# Assignment 4

## Topological Approach

#### Importing Libraries 
And we define our `DAG` variable, `g`.

In [1]:
import networkx as nx 
import numpy as np
import timeit
import sys
g = nx.DiGraph()
o=0

#### Defining Gate functions
Here we have defined all the functions we will use for our gates.



In [2]:
def AND(x,y):
    return x*y

In [3]:
def OR(x,y):
    if x+y >=1:
        return 1
    else:
        return 0

In [4]:
def NOT(x):
    if x==0:
        return 1
    else:
        return 0

In [5]:
def NOR(x,y):
    return NOT(OR(x,y))

In [6]:
def NAND(x,y):
    return NOT(AND(x,y))

In [7]:
def XOR(x,y):
    if x==y:
        return 0
    else:
        return 1

In [8]:
def XNOR(x,y):
    return NOT(XOR(x,y))

In [9]:
def BUF(x):
    return x

#### Function to read the netlist
Here we add the edges and corresponding node attributs to our `DAG` variable, `g`.

We use the list `connections` to add tuples which contain our directed edges, then we pass it to `g`.
We also use `gates` and `nodes` dictionaries to add attributes to `g`.

We have also added a `cycle` variable to check for a cycle in the graph `g`, which will make it unsolvable.
It will throw an error if there is a cycle present, like in `c17_1.net` netlist.

In [10]:
def read_file_circuit(file):
    f1=open(file,'r')
    lines=f1.readlines()
    for l in lines:
        if l.split()[1] == 'inv' or l.split()[1] == 'buf':
            connections.append((l.split()[2],l.split()[3]))
            gates[l.split()[3]]=l.split()[1]
            nodes[l.split()[3]]=l.split()[0]
        else:
            connections.append((l.split()[2],l.split()[4]))
            connections.append((l.split()[3],l.split()[4]))
            gates[l.split()[4]]=l.split()[1]
            nodes[l.split()[4]]=l.split()[0]
            
    g.add_edges_from(connections)
    for l in connections:
        if list(g.predecessors(l[0])) == []:
            gates[l[0]]="PI"
        #elif list(g.successors(l[1])) == []:
            #gates[l[1]]="PO"
    nx.set_node_attributes(g,gates,name="gate_type")
    nx.set_node_attributes(g,nodes,name="node")
    cycle=nx.algorithms.dag.is_directed_acyclic_graph(g)
    if cycle is False:
        print("There is a cycle present, netlist cannot be evaluated")
        sys.exit()
        f1.close()
    return

#### Function to read the input file
We return `inarr` which is the array with all primary inputs.

In [11]:
def read_file_inputs(file):
    h={}
    f1=open(file,'r')
    lines = f1.readlines()
    o = len(lines)-1
    head=list(lines[0].split())
    inarr=np.zeros((len(lines)-1,len(head)))
    j=k=0
    for i in lines:
        if k>0:
            for l in range(len(head)):
                inarr[j-1][l]=int(i.split()[l])
        j+=1
        k+=1
    f1.close()
    return inarr,head,o

#### Final Solver
This block contains the function `topo_solve`, which evaluates the state of the net according to the gate.

The states are recorded in a dictionary called `values` which is overwritten again and again to evaluate for all time instances. It is then sorted and appended to the `output` variable. `output` is then returned to the main function.

In [12]:
def topo_solve(file_net,file_input):
    temp=[]
    inarr,head,o=read_file_inputs(file_input)
    read_file_circuit(file_net)
    nl = list(nx.lexicographical_topological_sort(g))
    for i in range(o):
        values={}
        for l in nl:
            u=list(g.predecessors(l))
            if len(u)==2:
                n1=int(values.get(u[0]))
                n2=int(values.get(u[1]))
            elif len(u)==1:
                n1=int(values.get(u[0]))
            if gates.get(l) == 'PI':
                for k in range(len(head)):
                    if head[k]==l:
                        values[l]=int(inarr[i][k])
            elif gates.get(l) == 'nand2':
                values[l]=NAND(n1,n2)
            elif gates.get(l) == 'and2':
                values[l]=AND(n1,n2)
            elif gates.get(l) == 'or2':
                 values[l]=OR(n1,n2)
            elif gates.get(l) == 'inv':
                values[l]=NOT(n1)
            elif gates.get(l) == 'nor2':
                values[l]=NOR(n1,n2)
            elif gates.get(l) == 'xor2':
                values[l]=XOR(n1,n2)
            elif gates.get(l) == 'xnor2':
                values[l]=XNOR(n1,n2)
            elif gates.get(l) == 'buf':
                values[l]=BUF(n1)
        output.append(list(dict(sorted(values.items())).values()))
    return output,o

#### Final Evaluation and Output File
We are returned the `output` variable, that is ordered in ascending `ASCII` values, which is more than enough to sort our outputs.

We write into a file using a `for` loop and each file is uniquely named for each netlist.

In [13]:
start_time=timeit.default_timer()
connections=[]
gates={}
nodes={}
file_net='c432.net' # change file name here for different netlists.
file_input='c432.inputs'
output=[]
k=0
_,head,_=read_file_inputs(file_input)
output,o=topo_solve(file_net,file_input)
nl = list(nx.lexicographical_topological_sort(g))
f1=open("Output_topological_"+str(file_net),'w')
for l in sorted(nl):
    f1.write(str(l)+"\t")
f1.write("\n")
for k in range(o):
    for j in range(len(output[k])):
        f1.write(str(output[k][j]) + "\t")
    f1.write("\n")    
f1.close()
end_time=timeit.default_timer()
print("Time for topological is %s",(end_time-start_time))

Time for topological is %s 0.05172691121697426


#### Time taken: Topological
The time taken is recorded using `default_timer` function from `timeit` library. It records the time taken to complete from the start to the end of the main function.

# Event-Driven

#### Importing Libraries
First we import `collections` library which has the `deque` function, which we will use for queueing in our approach.

We initialize `q` which is our queue variable, and `output1` and `valuest` variables which have the same meaning as the topological approach but for event-driven.

In [14]:
from collections import deque
q=deque()
output1=[]
valuest={}

#### Function to solve for a net recursively
Here we have defined `val` function, which first checks the predecessors of the given input. 

Based on that it is either:
* Evaluated - *if it has evaluated predecessors*
* poped and appended - *if either of its predecessors are not evaluated*
* It's successors are appended - *if it is a primary input*

This logic works quite well, to evaluate multiple instances of inputs continuosly, it also checks if a particular net's value has not changed from the previous instance. Therefore discarding that node and any successors from the queue if the condition is satisified. This saves time and computing power.

In [15]:
def val(l):
    k=n1=n2=0
    u=list(g.predecessors(l))
    if len(u)==2:
        n1=valuest.get(u[0])
        n2=valuest.get(u[1])
    elif len(u)==1:
        n1=valuest.get(u[0])
    elif len(u)==0:
        f=list(g.successors(l))
        for a in f:
            q.append(a)
        q.popleft()
        return
    if n1 is None or n2 is None:
        q.append(l)
        q.popleft()  
        return
    else:
        if gates.get(l) == 'nand2':
            k=NAND(n1,n2)
        elif gates.get(l) == 'and2':
            k=AND(n1,n2)
        elif gates.get(l) == 'or2':
             k=OR(n1,n2)
        elif gates.get(l) == 'inv':
            k=NOT(n1)
        elif gates.get(l) == 'nor2':
            k=NOR(n1,n2)
        elif gates.get(l) == 'xor2':
            k=XOR(n1,n2)
        elif gates.get(l) == 'xnor2':
            k=XNOR(n1,n2)
        elif gates.get(l) == 'buf':
            k=BUF(n1)
        if k!=valuest.get(l):
            valuest[l]=k
            f=list(g.successors(l))
            for a in f:
                q.append(a)
        q.popleft()
    return

#### Initializing the variables
This block initializes the queue with **Primary Inputs**, and solve for the first instance of inputs. This initializes our `valuest` variable with values.

In [16]:
for l in sorted(gates):
    if gates.get(l)=='PI':
        q.append(l)
inarr1,head,o=read_file_inputs(file_input)
i=0
for l in q:
    valuest[l]=int(inarr1[0][i])
    i+=1
while(q):
    val(q[0])
output1.append(list(dict(sorted(valuest.items())).values()))

#### Function for initializing queue for further computation
This block checks which of the primary inputs has changed from the previous instance and initialiazes and adds it to the queue.

In [17]:
def input_compare(inarr1,j):
    for i in range(len(inarr1[0])):
        if inarr1[j][i]!=inarr1[j-1][i]:
            q.append(head[i])
            valuest[head[i]]=int(inarr1[j][i])
    return 

#### Final Evaluation and Output File
This block is the main function and it keeps initializing the queue with changed **Primary Inputs** and evaluating for it.

The `output1` list is appended with the sorted `valuest` dictionary and this is written into the final file for the output.

In [18]:
start_time2=timeit.default_timer()
for k in range(1,o):
    input_compare(inarr1,k)
    while(q):
        val(q[0])
    output1.append(list(dict(sorted(valuest.items())).values()))
f2=open("Output_event_driven_"+str(file_net),'w')
for l in sorted(nl):
    f2.write(str(l)+"\t")
f2.write("\n")
for k in range(o):
    for j in range(len(output1[k])):
        f2.write(str(output1[k][j]) + "\t")
    f2.write("\n")    
f2.close()
end_time2=timeit.default_timer()
print("Time for event driven is",(end_time2-start_time2),"s")  

Time for event driven is 0.04985751863569021 s


#### Time taken: Event-Driven
The time taken is calculated similarly to the topological one.

## Which method to use?
The two methods we have used are:
* Topological: *Sort the nodes in a topological order and evaluate.*
* Event-Driven: *Use a FIFO Data Structure (in our case Queue)to evaluate.*

The **Topological** method yields the values as shown:

| Net | Time Taken | Number of instances |
| --- | ----------- | ------------------ |
| c17.net | 3.054ms | 10 |
| c8.net | 8.934ms | 5 |
| parity.net | 18.405ms | 5 |
| c432.net| 43.226ms | 100 |





The **Event Driven** method yields time taken as shown:
| Net | Time Taken | Number of instances |
| --- | ----------- | ------------------ |
| c17.net | 1.048ms | 10 |
| c8.net | 5.295ms | 5 |
| parity.net | 1.681ms | 5 |
| c432.net| 40.225ms | 100 |

We can see some patterns like **Event Driven** is much faster for `parity.net`.

Also for `c432.net` both methods are similar in time, with **Topological** being a bit slower.

As we can cleasrly see, *Event Driven* is faster than *Topological*. But there is no direct relation on the number of instances to the "speed" of the computation.

Topological method works in a way that is sequential and organised, but this method has to be done all over again for each instance of time.

Event Driven method works in way that only evaluates the immediate successors, but it is much faster for different instances as it only has to check the inputs that have changed.

So in a way the factors that affect the "speed" of computation, would be number of instances to evaluate, number of nodes, and also how many instances change each time.

We can safely assume that **Event Driven** method is better usually, but it still depends on the netlist ;).
